In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## ineffiencies but i don't care
# 1 running compress data and not saving it while training
# 2 rerunning PCA for each compoent, could just run once and seperate each PC

In [3]:
import numpy as np
import torch
import os
import joblib
from torch import nn, optim


In [4]:
from helper.cluster import Kmeans, Kmedians, DBScan
from helper.dimension_reduction import PCA, AutoEncoder

In [5]:
import boto3
import io

In [6]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('dataandmodel20240505')
obj = bucket.Object('train/array.npy')

In [7]:
response = obj.get()
body = response['Body'].read()
file_stream = io.BytesIO(body)
np_features = np.load(file_stream, allow_pickle=True)

In [8]:
string_data = np_features[:, -1]
np.save('trained/audio_index.npy', string_data)

In [9]:
np_features = np.delete(np_features, -1, axis = 1)

In [10]:
np_features = np_features.astype(np.float64)
features_mean = np.mean(np_features)
features_std = np.std(np_features)
std_np_features = (np_features - features_mean) / features_std

In [11]:
loss_function = nn.MSELoss()

for i in range(1, std_np_features.shape[1] - 1):
    pca = PCA(i, np_features)
    os.makedirs(f"trained/pca_c{i}/", exist_ok=True)
    joblib.dump(pca, f"trained/pca_c{i}/pca.pkl")
    print(f"PCA saved with {i} components")

for i in range(1, 4):
    autoencoder = AutoEncoder(latent_space=i)
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
    autoencoder.custom_train(std_np_features, 25, loss_function, optimizer)
    os.makedirs(f"trained/auto_l{i}/", exist_ok=True)
    joblib.dump(autoencoder, f"trained/auto_l{i}/autoencoder.pkl")

PCA saved with 1 components
PCA saved with 2 components
PCA saved with 3 components
PCA saved with 4 components
PCA saved with 5 components
PCA saved with 6 components
PCA saved with 7 components
PCA saved with 8 components
PCA saved with 9 components
PCA saved with 10 components
PCA saved with 11 components


In [ ]:
for dirpath, dirnames, filenames in os.walk("trained"):
    for filename in filenames:
        if filename.endswith('.pkl'):
            file_path = os.path.join(dirpath, filename)
            model = joblib.load(filename=file_path)
            if filename.startswith("pca"):
                components = model.get_pca()

                for clusters in range(2, 6):
                    kmeans = Kmeans(clusters, 1000)
                    kmeans.fit(components)
                    os.makedirs(f"{dirpath}/kmeans_c{clusters}", exist_ok=True)
                    joblib.dump(kmeans, f"{dirpath}/kmeans_c{clusters}/cluster.pkl")

                    kmedians = Kmedians(clusters, 1000)
                    kmedians.fit(components)
                    os.makedirs(f"{dirpath}/kmedians_c{clusters}", exist_ok=True)
                    joblib.dump(kmedians, f"{dirpath}/kmedians_c{clusters}/cluster.pkl")
            elif filename.startswith("auto"):
                latent_space = model.compress_data(std_np_features)

                latent_space = latent_space.detach().cpu().numpy()
                for clusters in range(2, 6):
                    kmeans = Kmeans(clusters, 1000)
                    kmeans.fit(latent_space)

                    os.makedirs(f"{dirpath}/kmeans_c{clusters}", exist_ok=True)
                    joblib.dump(kmeans, f"{dirpath}/kmeans_c{clusters}/cluster.pkl")

                    kmedians = Kmedians(clusters, 1000)
                    kmedians.fit(latent_space)

                    os.makedirs(f"{dirpath}/kmedians_c{clusters}", exist_ok=True)
                    joblib.dump(kmedians, f"{dirpath}/kmedians_c{clusters}/cluster.pkl")

/Users/joshuascomputer/opt/anaconda3/envs/thesis_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/joshuascomputer/opt/anaconda3/envs/thesis_env/lib/python3.8/site-packages/numpy/core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [ ]:
for clusters in range(2, 6):
    kmeans = Kmeans(clusters, 1000)
    kmeans.fit(np_features)

    os.makedirs(f"trained/base/kmeans_c{clusters}", exist_ok=True)
    joblib.dump(kmeans, f"trained/base/kmeans_c{clusters}/cluster.pkl")

    kmedians = Kmedians(clusters, 1000)
    kmedians.fit(np_features)

    os.makedirs(f"trained/base/kmedians_c{clusters}", exist_ok=True)
    joblib.dump(kmedians, f"trained/base/kmedians_c{clusters}/cluster.pkl")

In [ ]:
## take a sample reduce it
